In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold

resultsWin = pd.read_csv('data/RegularSeasonDetailedResults.csv')
teams = pd.read_csv('data/Teams.csv')

# Get only winning teams game stats to predict their score
resultsWin = resultsWin.drop(['WTeamID', 'LTeamID', 'WLoc'], axis=1)

def neuralNetwork(results) :
    train_features, test_features, train_outcome, test_outcome = train_test_split(
        results.drop("WScore", axis=1),
        results.WScore,
        test_size=0.30, 
        random_state=11
    )
    scaler = MinMaxScaler()
    mlp_reg = MLPClassifier()

    imputer = Imputer()
    selector = SelectPercentile()
    threshold = VarianceThreshold(.1)
    pipe = make_pipeline(imputer, threshold, selector, scaler, mlp_reg)

    param_grid = {
        'selectpercentile__percentile':range(10, 30, 5)
        }

    crossVal = KFold()
    grid = GridSearchCV(pipe, param_grid, cv = crossVal, scoring="neg_mean_absolute_error")
    grid.fit(train_features, train_outcome)
    grid.score(test_features, test_outcome)

    score = grid.score(test_features, test_outcome)

    predictedValues = grid.predict(test_features)

    return [score, predictedValues, grid, test_outcome]

neural = neuralNetwork(resultsWin)

# How many points off were we from predicting the winning score?
print(neural[0])

# This can be exactly replicated for the other team using LScore for losing score
# Then need to randomize which teams get which grid and we run the grid for each playoff game invididually
# in their own csv. Take the two scores, see who won, move them manually to the next round.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imp

In [ ]:
resultsWin.head()

I'm not quite done withe the following but it is a way to predict the outcomes of the tournament games as they contunue. The games each round can be based off of the predicted outcome of the last round.

In [ ]:
def next_round(teams_df):
    '''Takes a dataframe with two columns: School id, and ranking, none of 
    these teams should have been eliminated. It returns the next matchups for these 
    teams based on their ranking'''
    arar = np.char.array(['01','16','08','09','05','12','04','13','06','11','03','14','07','10','02','15'])
    arr = np.append(arar, arar)
    first_round_bracket = np.char.array(['W', 'X', 'Y', 'Z']).repeat(16) + np.append(arr, arr)
    if(len(teams_df) < 64):
        won_ranks = teams_df['Seed'].values
        first_round_bracket = np.array([x for x in first_round_bracket if x in won_ranks])
    #print(first_round_bracket)
    picks1 = first_round_bracket[np.arange(1, len(teams_df), 2)]
    picks2 = first_round_bracket[np.arange(0, len(teams_df), 2)]
    #print(picks1)
    #print(picks2)
    teams_df = teams_df.set_index('Seed')
    teams_next = pd.DataFrame()
    teams_next['team1'] = teams_df.loc[picks1, 'TeamID'].values
    teams_next['team2'] = teams_df.loc[picks2, 'TeamID'].values
    teams_next['rank1'] = picks1
    teams_next['rank2'] = picks2
    return teams_next

def find_winners(nx):
    '''Takes a dataframe with teamsids and ranks from matches and keeps only the 
    winners'''
    nx['score'] = (nx['team1'] - nx['team2'])
    nx['TeamID'] = nx.loc[:, 'team2']
    nx['Seed'] = nx.loc[:, 'rank2']
    
    condition = nx['score'] > 0
    #this can be replaced with an ouput from a neural net to predict winners
    underdogs = nx.loc[condition,['rank1', 'team1']]
    underdogs.columns = ['Seed', 'TeamID']
    nx.update(underdogs)
    return(nx)#.loc[:, ['Seed', 'TeamID']])

def first_four(teams_df, games_record):
    pregames = teams_df.loc[teams_df['Seed'].str.contains('a|b'),:]
    teams_df = teams_df.loc[~teams_df['Seed'].str.contains('a|b'),:]
    #features = pd.merge(games, team_summary_stats, how='left', left_on=['team1'], right_on=['TeamID'])
    #features = pd.merge(games, team_summary_stats, how='left', left_on=['team2'], right_on=['TeamID'], suffixes=('', '_t2'))
    teams_next = pd.DataFrame()
    teams_next['team1'] = pregames.iloc[np.arange(1, len(pregames), 2), 1].values
    teams_next['team2'] = pregames.iloc[np.arange(0, len(pregames), 2), 1].values
    teams_next['rank1'] = pregames.iloc[np.arange(1, len(pregames), 2), 0].values
    teams_next['rank2'] = pregames.iloc[np.arange(0, len(pregames), 2), 0].values
    to_begin = find_winners(teams_next.copy())
    to_begin['round'] = 0
    if len(games_record) < 1:
        games_record = to_begin.copy()#.loc[:,:]
    else:
        games_record = games_record.append(to_begin.copy(), ignore_index=True)
    to_begin['Seed'] = [x[0:-1] for x in to_begin['Seed'].values]
    teams_df = teams_df.append(to_begin.loc[:, ['Seed', 'TeamID']])
    #print(to_begin.loc[:, ['Seed', 'TeamID']])
    return([teams_df, games_record])

games_record = pd.DataFrame() # records all matches
teams_df = pd.read_csv('data/NCAATourneySeeds.csv')
teams_df = teams_df.loc[teams_df.Season == 2003, ['Seed', 'TeamID']]
teams_df, games_record = first_four(teams_df, games_record)
#print(teams_df)
round = 1 

#runs until only 1 team remains
while len(teams_df) > 1:
    games = next_round(teams_df)
    features = pd.merge(games, team_summary_stats, how='left', left_on=['team1'], right_on=['TeamID'])
    features = pd.merge(features, team_summary_stats, how='left', left_on=['team2'], right_on=['TeamID'], suffixes=('', '_t2'))
    print(features.head(2))
    games['round'] = round
    teams_df = find_winners(games)
    if len(games_record) < 1:
        games_record = games#.loc[:,:]
    else:
        games_record = games_record.append(games, ignore_index=True)
    round = round + 1
#print(games_record)

In [6]:
regular_season_results = pd.read_csv('data/RegularSeasonDetailedResults.csv')
post_season_outcomes = pd.read_csv('data/NCAATourneyDetailedResults.csv')

Creating a set of regular season features

In [7]:
winners = regular_season_results.loc[:,['Season', 'WTeamID', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
                                         'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
losers = regular_season_results.loc[:,['Season', 'LTeamID', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
                                      'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
winners.columns = ['Season', 'TeamID', 'FGM', 'FGA', 'FGM3', 'FGA3',
                                      'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
losers.columns = ['Season', 'TeamID', 'FGM', 'FGA', 'FGM3', 'FGA3',
                                      'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
all_teams = winners.copy()
all_teams = all_teams.append(losers.copy(), ignore_index=True)
team_summary_stats = all_teams.groupby(['Season', 'TeamID'], as_index=False).mean()
print(team_summary_stats.head())
team_summary_stats.to_csv('data/team_summary_stats.csv')

   Season  TeamID        FGM        FGA      FGM3       FGA3        FTM  \
0    2003    1102  19.142857  39.785714  7.821429  20.821429  11.142857   
1    2003    1103  27.148148  55.851852  5.444444  16.074074  19.037037   
2    2003    1104  24.035714  57.178571  6.357143  19.857143  14.857143   
3    2003    1105  24.384615  61.615385  7.576923  20.769231  15.423077   
4    2003    1106  23.428571  55.285714  6.107143  17.642857  10.642857   

         FTA         OR         DR        Ast         TO       Stl       Blk  \
0  17.107143   4.178571  16.821429  13.000000  11.428571  5.964286  1.785714   
1  25.851852   9.777778  19.925926  15.222222  12.629630  7.259259  2.333333   
2  20.928571  13.571429  23.928571  12.107143  13.285714  6.607143  3.785714   
3  21.846154  13.500000  23.115385  14.538462  18.653846  9.307692  2.076923   
4  16.464286  12.285714  23.857143  11.678571  17.035714  8.357143  3.142857   

          PF  
0  18.750000  
1  19.851852  
2  18.035714  
3  20.23

Creating a set of post season outcomes to test with

In [ ]:
post_season_outcomes.head(2)

In [17]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
post_season_outcomes['score_dif'] = post_season_outcomes.WScore - post_season_outcomes.LScore
outcome = post_season_outcomes.loc[:,['Season', 'WTeamID', 'LTeamID', 'score_dif']]
mixing_matrix = np.random.choice([True, False], len(outcome))
mixed_outcome = outcome.copy()
print(mixed_outcome.head())
mixed_outcome.loc[mixing_matrix, ['WTeamID', 'LTeamID']] = mixed_outcome.loc[mixing_matrix, ['LTeamID', 'WTeamID']].values 
mixed_outcome.loc[mixing_matrix, ['score_dif']] = mixed_outcome.loc[mixing_matrix, ['score_dif']].mul(-1)
print(mixed_outcome.head())
print(len(mixed_outcome))
features = pd.merge(mixed_outcome, team_summary_stats, left_on=['WTeamID', 'Season'], right_on=['TeamID', 'Season'])
print(len(features))
features = pd.merge(features, team_summary_stats, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], suffixes=('', '_t2'))
print(len(features))
features.loc[:, 'FGM':'PF'] = features.loc[:, 'FGM':'PF'].values - features.loc[:, 'FGM_t2':'PF_t2'].values
features = features.loc[:, 'Season':'PF']
features.to_csv('data/mixed_prepared_features.csv')
print(features.head(15))
print(team_summary_stats.head(15))

   Season  WTeamID  LTeamID  score_dif
0    2003     1421     1411          8
1    2003     1112     1436         29
2    2003     1113     1272         13
3    2003     1141     1166          6
4    2003     1143     1301          2
   Season  WTeamID  LTeamID  score_dif
0    2003     1421     1411          8
1    2003     1436     1112        -29
2    2003     1272     1113        -13
3    2003     1141     1166          6
4    2003     1143     1301          2
1048
1048
1048
    Season  WTeamID  LTeamID  score_dif  TeamID       FGM        FGA  \
0     2003     1421     1411          8    1421 -0.354023   1.526437   
1     2003     1421     1400        -21    1421 -3.620690  -5.635468   
2     2003     1277     1400         -9    1277 -4.967742 -11.331797   
3     2003     1345     1400        -10    1345 -4.250000  -8.035714   
4     2003     1163     1400         -4    1163  1.533333  -0.228571   
5     2003     1436     1112        -29    1436 -5.493842  -9.852217   
6     2003   

### A possible model for predicting games based on regular season summary statistics

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from ModelFunctions import DecisionTreeFunc, KnnFunc, BayesianRidge, NeuralNetworkFunc

import warnings
#warnings.filterwarnings('ignore') # disables warnings
warnings.filterwarnings('ignore')
#features.score_dif = 1*(features.score_dif > 0)
test_year = features.loc[features.Season == 2018, :]
test_features = features.loc[features.Season != 2018, :]

In [ ]:
model = KnnFunc(test_features.drop(columns=['score_dif', 'Season']), test_features.score_dif)

In [26]:
model.predict(test_year.drop(columns=['score_dif', 'Season']))

array([ 18.,   4., -12., -27.,  12.,  -9.,  -2.,  34.,  22.,  -2.,   2.,
       -39.,  12.,  19.,   2., -26.,  -4.,   4.,  16.,  17.,  13.,  10.,
       -10.,  33.,   8.,  -2.,  -2.,   5.,  -4.,  -4.,  -5.,   4.,  23.,
        -8., -20., -17.,  12.,   3.,  35., -14.,  15., -39., -18.,  39.,
       -10.,  23.,  -1.,   8.,  -1., -23.,  -9.,  15., -22.,   1., -28.,
         9., -13.,   4.,   2.,   8.,  19.,  13.,   7.,   6.,  13.,   3.,
        -5.])

In [88]:
scaler = MinMaxScaler()
mlp_reg = MLPRegressor()
print(features.head())
param_grid = {}#'mlpregressor__activation' : ['identity', 'logistic', 'tanh', 'relu']}
#print(test_year.score_dif)
pipe = make_pipeline(scaler, mlp_reg)
grid = GridSearchCV(pipe, param_grid)
grid.fit(test_features.drop(columns=['score_dif', 'Season']), test_features.score_dif)
#print(grid.score(test_year.drop(columns=['score_dif', 'Season']), test_year.score_dif))

   Season  WTeamID  LTeamID  score_dif  TeamID        FGM        FGA  \
0    2003     1421     1411          8    1421  24.379310  56.793103   
1    2003     1112     1436         29    1112  30.321429  65.714286   
2    2003     1112     1211          1    1112  30.321429  65.714286   
3    2003     1153     1211         -5    1153  22.892857  56.678571   
4    2003     1112     1242         -3    1112  30.321429  65.714286   

       FGM3       FGA3        FTM    ...        FGA3_t2     FTM_t2     FTA_t2  \
0  6.482759  18.000000  15.965517    ...      18.500000  17.400000  28.066667   
1  7.035714  20.071429  17.535714    ...      15.482759  12.862069  19.551724   
2  7.035714  20.071429  17.535714    ...      19.064516  17.774194  24.645161   
3  6.678571  19.500000  14.857143    ...      19.064516  17.774194  24.645161   
4  7.035714  20.071429  17.535714    ...      14.133333  16.066667  24.133333   

       OR_t2      DR_t2     Ast_t2      TO_t2     Stl_t2    Blk_t2      PF_t2  


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-pa

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('mlpregressor', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
results = pd.DataFrame()
results['real'] = test_year.score_dif
results['predicted'] = model.predict(test_year.drop(columns=['score_dif', 'Season']))
results['FGAdif'] = test_year.FGA

results[results['real'] * results['predicted'] > 0]
results['correct'] = results['real'] * results['predicted'] > 0
display(results)
len(results[results['real'] * results['predicted'] > 0]) / len(results)

,real,predicted,FGAdif,correct
981,-10,18.0,3.519886,False
982,7,4.0,-3.531250,True
983,-15,-12.0,-2.687500,True
984,-4,-27.0,5.833822,True
985,-2,12.0,2.303030,False
986,18,-9.0,1.596774,False
987,-3,-2.0,1.387868,True
988,-21,34.0,-7.758467,False
989,20,22.0,-7.199643,True
990,-22,-2.0,-0.818182,True


0.5970149253731343

In [2]:
print(features)

NameError: name 'features' is not defined